In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f urls-2020.txt || wget https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/urls-2020.txt")

In [ ]:
urls = open("urls-2020.txt").read().split("\n")

In [ ]:
import os
if not os.path.isdir("artikel"):
    os.mkdir("artikel")

In [ ]:
def article_filename(url):
    c = url.split("/")
    return "/".join(["artikel", c[-1]])

In [ ]:
!pip install tqdm

In [ ]:
import requests
from tqdm.auto import tqdm

s = requests.Session()
for u in tqdm(urls):
    if u == '':
        continue
    filename = article_filename(u)
    if not os.path.isfile(filename):
        print(filename)
        print(u)
        r = s.get("https://www.heise.de" + u)
        open(filename, 'wb').write(r.content)    

In [ ]:
import json
from bs4 import BeautifulSoup
res = []
for u in tqdm(urls):
    filename = article_filename(u)
    html = open(filename).read()
    soup = BeautifulSoup(html)
    d = {}
    d["title"] = soup.h1.text.strip()
    d["header"] = soup.select_one("#meldung > div.article-layout__header-container > header > p").text.strip()
    d["author"] = soup.select_one("a.redakteurskuerzel__link").attrs["title"]
    d["text"] = "\n".join([p.text.strip() 
                            for p in soup.select("#meldung > div.article-layout__content-container > div > p")])
    d["keywords"] = soup.find("meta", {"name": "keywords"})["content"]
    ld = json.loads(soup.find("script", type="application/ld+json").string)
    for k in ["identifier", "url", "datePublished", "commentCount"]:
        d[k] = ld[k]
    
    res.append(d)

In [ ]:
res = []
for u in tqdm(urls):
    try:
        filename = article_filename(u)
        html = open(filename).read()
        soup = BeautifulSoup(html)
    except:
        # Datei nicht gefunden, invalides HTML etc.
        continue
    d = {}
    d["title"] = soup.h1.text.strip()
    try:
        d["header"] = soup.select_one("#meldung > div.article-layout__header-container > header > p").text.strip()
    except:
        pass
    try:
        d["author"] = soup.select_one("a.redakteurskuerzel__link").attrs["title"]
    except:
        pass
    d["text"] = "\n".join([p.text.strip() 
                            for p in soup.select("#meldung > div.article-layout__content-container > div > p")])
    try:
        d["keywords"] = soup.find("meta", {"name": "keywords"})["content"]
    except:
        pass
    try:
        ld = json.loads(soup.find("script", type="application/ld+json").string)
        for k in ["identifier", "url", "datePublished", "commentCount"]:
            d[k] = ld[0][k]
    except:
        pass
    
    res.append(d)

In [ ]:
import pandas as pd
articles = pd.DataFrame(res)
len(articles)

In [ ]:
articles[articles["identifier"].isna()]

In [ ]:
articles[articles["url"].isna()]

In [ ]:
articles = articles.dropna(subset=["identifier"])
len(articles)

In [ ]:
articles = articles.drop_duplicates(subset=["identifier"]).set_index("identifier")
articles.index = articles.index.astype(int)
len(articles)

In [ ]:
import sqlite3
sql = sqlite3.connect("heise-articles-2020.db")
articles.to_sql("articles", sql, index_label="id", if_exists="replace")